# Computer Vision Project Notebook
## Fighting fire with fire: Detecting synthetic AI-generated images

By:
Adel del Valle -

Adith Santhosh -

Joshua Alfred Jayapal - jj3811



## Importing libraries

In [1]:
%pip install tensorflow

## Pre-requisite

Download the datasets CIFAKE and COCO-fake at these links and keep them in your repo directory

CIFAKE: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images

COCO-fake: https://github.com/aimagelab/COCOFake

## Unpacking (only CIFAKE for now)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

# Path to your zip file
zip_file_path = '/content/drive/MyDrive/CV_Project_F24_Data/cifake.zip'

# Destination folder in colab
extract_path = '/content/CV_project_F24'


if os.path.exists(zip_file_path):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Successfully extracted {zip_file_path} to {extract_path}")
    except zipfile.BadZipFile:
        print(f"Error: Invalid zip file: {zip_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print(f"Error: Zip file not found at {zip_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully extracted /content/drive/MyDrive/CV_Project_F24_Data/cifake.zip to /content/CV_project_F24


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')


# !cp -r '/content/drive/MyDrive/CV_Project_F24_Data' /content/

Mounted at /content/drive
^C


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from timeit import default_timer as timer
from torchvision.models import swin_v2_t, Swin_V2_T_Weights, efficientnet_v2_s, EfficientNet_V2_S_Weights
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [5]:
path = "/content/CV_project_F24/cifake-real-and-ai-generated-synthetic-images/versions/3"
train_dir = os.path.join(path, "train")
test_dir = os.path.join(path, "test")
epochs = 50
batch_size = 128
lr = 0.001

In [6]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)



test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

train = train_datagen.flow_from_directory(train_dir,
                                          target_size=(224, 224),
                                          batch_size=batch_size,
                                          class_mode='binary',
                                          subset='training')

val = train_datagen.flow_from_directory(train_dir,
                                      target_size=(224, 224),
                                      batch_size=batch_size,
                                      class_mode='binary',
                                      subset='validation')

test = test_datagen.flow_from_directory(test_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode='binary')

Found 80000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [7]:
train_dataset = tf.data.Dataset.from_generator(lambda: train,
                                               output_signature=(
                                                   tf.TensorSpec(shape=(None, 224, 244, 3), dtype=tf.float32),
                                                   tf.TensorSpec(shape=(None,), dtype=tf.float32)))

val_dataset = tf.data.Dataset.from_generator(lambda: val,
                                               output_signature=(
                                                   tf.TensorSpec(shape=(None, 224, 244, 3), dtype=tf.float32),
                                                   tf.TensorSpec(shape=(None,), dtype=tf.float32)))

test_dataset = tf.data.Dataset.from_generator(lambda: test,
                                              output_signature=(
                                                  tf.TensorSpec(shape=(None, 224, 244, 3), dtype=tf.float32),
                                                  tf.TensorSpec(shape=(None,), dtype=tf.float32)))

In [8]:
model = keras.applications.EfficientNetV2B2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
    name="efficientnetv2-b2",
)
model.trainable = False
model = tf.keras.Sequential([
    model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b2 (Functional)       │ (None, 7, 7, 1408)          │       8,769,374 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1408)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           1,409 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,770,783 (33.46 MB)

 Trainable params: 1,409 (5.50 KB)

 Non-trainable params: 8,769,374 (33.45 MB)

In [ ]:
# train model

steps_per_epoch = train.samples // train.batch_size
test_steps = test.samples // test.batch_size

model.fit(train, epochs=20, steps_per_epoch=steps_per_epoch, validation_data=val)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


121/625 ━━━━━━━━━━━━━━━━━━━━ 15:09 2s/step - accuracy: 0.5029 - loss: 0.6952

In [ ]:
import matplotlib.pyplot as plt

history = model.history

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# prompt: save the trained model

model.save('base_model_effnetv2b2_2.keras')

In [ ]:
Y_pred = model.predict(test_dataset)
y_pred = np.round(Y_pred).astype(int)

# plot confusion matrix and roc curve


cm = confusion_matrix(test.classes, y_pred)
print(cm)

fpr, tpr, thresholds = roc_curve(test.classes, y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)

